# Short path, Reconstructing the path
In this assignment you will use Spark to compute the shortest path between two vertices. In the video, you have learned how to compute the distances between a source vertex and all other vertices in a graph. Now, your task is to reconstruct the shortest path, that is a sequence of vertices connected by the edges.

Dataset location: /data/twitter/twitter_sample_small.txt

Format: user_id \t follower_id

You can start with the code described in "Starter..." (see the next self-reading).

Your task is to find the shortest path between vertices 12 and 34. In case of multiple shortest paths (that is, disjoint paths with the same length), any will suffice. Output format is the sequence of vertices, delimited by a comma, without spaces. For example, the path “12 -> 42 -> 34” should be printed as:
    
12,42,34

Hint: before submitting, check your stopping criteria. In BFS, the search was exhaustive, and in this task your program may terminate earlier, thus saving some precious time.

The result on the sample dataset:
    
12,422,53,52,107,20,23,274,34

If you want to deploy the environment on your own machine, please use bigdatateam/all-spark-with-data Docker container. New image: bigdatateam/hysh-full:py3-c1

In [ ]:
from pyspark import SparkConf, SparkContext
import re

In [2]:
try:
    sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local").set("spark.cores.max", "16"))
except:
    pass

In [ ]:
dataAddress = "/data/twitter/twitter_sample_small.txt"
n = 10  
start, end = 12, 34

In [20]:
def parse_edge(s):
    user, follower = s.split("\t")
    return (int(follower), int(user))

def step(item):
    prev_v, prev_d, next_v = item[0], item[1][0], item[1][1]
    return (next_v, prev_d + [next_v])

def found():
    return  paths.filter(lambda x: x[0] == end).count()

forward_edges = sc.textFile(dataAddress).map(parse_edge).cache().partitionBy(n).persist()

In [23]:

paths = sc.parallelize([(start, [start])]).partitionBy(n)


while not found():
    paths = paths.join(forward_edges, n).map(step).cache()

final_paths = (paths.filter(lambda value: value[0] == end)
                    .map(lambda value: value[1])
                   ).cache()

result = final_paths.take(1)[0]

print(','.join(map(str,result)))

12,422,53,52,107,20,23,274,34
